In [55]:
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib
import torch.nn as nn

In [56]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout=0.5):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size, 64)  # Учитываем bidirectional
        self.fc2 = nn.Linear(64, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, self.hidden_size).to(x.device)  # Учитываем bidirectional
        c0 = torch.zeros(self.num_layers, self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, :]  # Берём последний временной шаг
        out = self.dropout(out)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [57]:
model = torch.load('lstm_model.pth')
scaler_features = joblib.load('scaler_features.pkl')
scaler_target = joblib.load('scaler_target.pkl')

<ipython-input-57-c90eedb96ef9>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('lstm_model.pth')


In [58]:
data = pd.read_csv("SBER(2 month).csv")
data = data.replace('nan', np.nan)
data['Объём'] = data['Объём'].fillna(0)
data = data.iloc[::-1].reset_index(drop=True)
data

,Дата,Цена,Откр.,Макс.,Мин.,Объём,Изм. %
0,26.11.2024,"223,16","227,80","231,90","221,20","96,80M","-2,04%"
1,27.11.2024,"226,30","223,90","226,47","219,20","112,46M","1,41%"
2,28.11.2024,"228,56","227,00","229,50","222,48","67,84M","1,00%"
3,29.11.2024,"236,49","228,48","236,99","227,55","70,30M","3,47%"
4,02.12.2024,"235,17","237,02","238,70","234,32","45,14M","-0,56%"
5,03.12.2024,"230,80","235,29","235,97","230,26","45,32M","-1,86%"
6,04.12.2024,"224,55","231,00","233,79","224,30","72,42M","-2,71%"
7,05.12.2024,"233,48","224,57","233,75","222,82","86,99M","3,98%"
8,06.12.2024,"237,84","233,63","238,25","231,52","81,10M","1,87%"
9,09.12.2024,"237,29","239,29","240,51","236,75","52,49M","-0,23%"


In [59]:
for i in range(len(data)):
  data.loc[i, "Цена"] = data.loc[i, "Цена"].replace(",", ".")
  data.loc[i, "Откр."] = data.loc[i, "Откр."].replace(",", ".")
  data.loc[i, "Макс."] = data.loc[i, "Макс."].replace(",", ".")
  data.loc[i, "Мин."] = data.loc[i, "Мин."].replace(",", ".")
  data.loc[i, "Изм. %"] = data.loc[i, "Изм. %"].replace(",", ".")
  data.loc[i, "Изм. %"] = data.loc[i, "Изм. %"].replace("%", "")
  data.loc[i, "Объём"] = str(data.loc[i, "Объём"]).replace(",", ".")
  if 'M' in  data.loc[i, "Объём"]:
   data.loc[i, "Объём"] = str(data.loc[i, "Объём"]).replace("M", "")
  elif 'B' in data.loc[i, "Объём"]:
    data.loc[i, "Объём"] = str(data.loc[i, "Объём"]).replace("B", "")

columns_to_convert = ['Цена', 'Откр.', 'Макс.', 'Мин.', 'Объём', 'Изм. %']
# Преобразуем строки в числа, заменяя некорректные значения на NaN
for col in columns_to_convert:
    data[col] = pd.to_numeric(data[col], errors='coerce')


In [60]:
data

,Дата,Цена,Откр.,Макс.,Мин.,Объём,Изм. %
0,26.11.2024,223.16,227.80,231.90,221.20,96.80,-2.04
1,27.11.2024,226.30,223.90,226.47,219.20,112.46,1.41
2,28.11.2024,228.56,227.00,229.50,222.48,67.84,1.00
3,29.11.2024,236.49,228.48,236.99,227.55,70.30,3.47
4,02.12.2024,235.17,237.02,238.70,234.32,45.14,-0.56
5,03.12.2024,230.80,235.29,235.97,230.26,45.32,-1.86
6,04.12.2024,224.55,231.00,233.79,224.30,72.42,-2.71
7,05.12.2024,233.48,224.57,233.75,222.82,86.99,3.98
8,06.12.2024,237.84,233.63,238.25,231.52,81.10,1.87
9,09.12.2024,237.29,239.29,240.51,236.75,52.49,-0.23


In [61]:
features = ['Откр.', "Макс.", "Мин.", "Объём", "Изм. %"]
target = 'Цена'

In [62]:
# Нормализация новых данных
scaled_features_data = scaler_features.transform(data[features])
scaled_target_data = scaler_target.transform(data[target].to_frame())

sequence_length = 20

# Подготовка последовательности
input_sequence = scaled_features_data[-sequence_length:]  # Берём последние 34 точки
input_sequence = input_sequence.mean(axis=0)  # Агрегируем по времени (среднее значение)
input_sequence = input_sequence.reshape(1, -1)  # Форма (1, num_features)

# Преобразование в тензор
input_tensor = torch.tensor(input_sequence, dtype=torch.float32)

# Предсказание
model.eval()
with torch.no_grad():
    predicted_scaled_price = model(input_tensor)

# Обратное преобразование
predicted_price = scaler_target.inverse_transform(predicted_scaled_price.numpy())
print(f"Предсказанная цена на понедельник: {predicted_price[0][0]:.2f}")

Предсказанная цена на понедельник: 283.01
